In [2]:
import os

In [4]:
#itersting thorugh my folder to find the corredct sub directory,
# and start of path setup for dictionary and sub folder organization 

#var steup 
data = 'brisc2025' #root 
path_segtask = 'brisc2025/segmentation_task/train' #seg path 
file_paths = [] #file path storage


for root, dirs, files in os.walk(data):
    for d in dirs: #dierectory iteration 
        directories = os.path.join(root, d) #join 
        if directories.startswith(path_segtask): #image iteration 
            for images in os.listdir(directories):
                if (images.endswith(".jpg")):
                    path = directories.replace("brisc2025/segmentation_task/train/", "") #removing full path for appened
                    img = images.replace(".jpg", "")
                    s = img.split('_')
                    key = "_".join(s[-3:])
                    a = path + "/" + key
                    #print(a)
                    if a in file_paths:
                       continue
                    else:
                       file_paths.append(a)
print(file_paths)
 

['images/pi_co_t1', 'images/pi_sa_t1', 'images/gl_co_t1', 'images/pi_ax_t1', 'images/me_co_t1', 'images/gl_ax_t1', 'images/gl_sa_t1', 'images/me_sa_t1', 'images/me_ax_t1', 'masks/pi_co_t1', 'masks/pi_sa_t1', 'masks/gl_co_t1', 'masks/pi_ax_t1', 'masks/me_co_t1', 'masks/gl_ax_t1', 'masks/gl_sa_t1', 'masks/me_sa_t1', 'masks/me_ax_t1']


In [ ]:
nameI = "images"

if os.path.isdir(nameI):
    print('image folder is already made')
else:
    os.mkdir(nameI)

image folder is already made
['gl_co_t1', 'gl_ax_t1', 'gl_sa_t1', 'pi_sa_t1', 'me_sa_t1', 'me_ax_t1', 'pi_ax_t1', 'pi_co_t1', 'me_co_t1']


In [39]:
nameM = "masks"

if os.path.isdir(nameM):
    print('mask folder is already made')
    print(os.listdir(nameM))
else:
    os.mkdir(nameM)

mask folder is already made
['gl_co_t1', 'gl_ax_t1', 'gl_sa_t1', 'pi_sa_t1', 'me_sa_t1', 'me_ax_t1', 'pi_ax_t1', 'pi_co_t1', 'me_co_t1']


In [42]:
import shutil

In [ ]:
#Trting to move files nito appropriate sub dierectors of new folders
base = 'brisc2025/segmentation_task/train/images'
seg_dirs = os.listdir(base)
ImgPaths = os.listdir(nameI)

for fp in file_paths:
    for dir_ in seg_dirs:
        for img in ImgPaths:
            if 'pi_co_t1' in dir_ and 'pi_co_t1' in img and 'pi_co_t1' in fp:
                print(dir_ + " " + img + " " + fp)
                shutil.move(dir_, fp)           
#print("After moving directories:")
#print(os.listdir(base))


brisc2025_train_04308_pi_co_t1.jpg pi_co_t1 images/pi_co_t1


FileNotFoundError: [Errno 2] No such file or directory: 'brisc2025_train_04308_pi_co_t1.jpg'